# Importing The Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Reading the Dataset (2018)
This notebook has done analysis on the 2018 happiness data which predicts the happiness score. 

In [ ]:
df=pd.read_csv("data/happ.csv")
df.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393
1,2,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,3,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,4,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,5,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357


In [ ]:
df.describe()

,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
count,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000
mean,78.500000,5.375917,0.891449,1.213237,0.597346,0.454506,0.181006,0.112449
std,45.177428,1.119506,0.391921,0.302372,0.247579,0.162424,0.098471,0.096343
min,1.000000,2.905000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,39.750000,4.453750,0.616250,1.066750,0.422250,0.356000,0.109500,0.051000
50%,78.500000,5.378000,0.949500,1.255000,0.644000,0.487000,0.174000,0.082000
75%,117.250000,6.168500,1.197750,1.463000,0.777250,0.578500,0.239000,0.139000
max,156.000000,7.632000,2.096000,1.644000,1.030000,0.724000,0.598000,0.457000


In [ ]:
df.columns

Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')

# Splitting Training Set and Test Set
Sincewe are going to predict the happiness score, we are going to define the output variable as the score and the input as all the remaining features of X.

In [ ]:
X = df.drop(["Score"],axis = 1)
y = df.Score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=1)

In [ ]:
X_train.head()

,Overall rank,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
66,67,Moldova,0.657,1.301,0.620,0.232,0.171,0.000
33,34,Singapore,1.529,1.451,1.008,0.631,0.261,0.457
141,142,Angola,0.730,1.125,0.269,0.000,0.079,0.061
59,60,Kazakhstan,1.143,1.516,0.631,0.454,0.148,0.121
127,128,Georgia,0.853,0.592,0.643,0.375,0.038,0.215


In [ ]:
y_train.head()

66     5.640
33     6.343
141    3.795
59     5.790
127    4.340
Name: Score, dtype: float64

# Identifying Missing Columns and Categorical Variables 

In [ ]:
cols_miss = [col for col in X_train.columns if X_train[col].isnull().any()]
cols_miss

[]

We see there are no missing values in the dataset.

In [ ]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
object_cols

['Country or region']

We see that the only feature that is categorical is the "country or religion". The given dataset is of 156 countries so we can drop this column since it is not effective to use a technique such as one-hot-encoding.Moreover the ranking of the countries is given in the dataset.

We can also drop the ranking of the country since it is only based on the score of happiness.The score of happiness is what we need to predict in the first place. Rather than keeping it in X we can drop it too.

In [ ]:
X_train = X_train.drop(["Country or region","Overall rank"],axis=1)
X_test = X_test.drop(["Country or region","Overall rank"],axis=1)

Note that we need to predict the happiness score. For a given country its name does not define how happy the country actually is.

In [ ]:
X_train

,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
66,0.657,1.301,0.620,0.232,0.171,0.000
33,1.529,1.451,1.008,0.631,0.261,0.457
141,0.730,1.125,0.269,0.000,0.079,0.061
59,1.143,1.516,0.631,0.454,0.148,0.121
127,0.853,0.592,0.643,0.375,0.038,0.215
...,...,...,...,...,...,...
133,0.131,0.867,0.221,0.390,0.175,0.099
137,0.793,1.413,0.609,0.163,0.187,0.011
72,1.039,1.498,0.700,0.307,0.101,0.154
140,0.472,1.215,0.079,0.423,0.116,0.112


# Model
There will be various models used here depending on tthe accuracy score we will be concluding which of them perform good over the test set that we split.

**Linear Regression**

In [ ]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

LinearRegression()

**Decision Tree**

In [ ]:
model2 = DecisionTreeRegressor(random_state = 0)
model2.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

**Random Forest**

In [ ]:
model3 = RandomForestRegressor(random_state = 0)
model3.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

**XGBoost**

In [ ]:
model4 = XGBRegressor()
model4.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

# Prediction and Accuracy

In [ ]:
pred1 = model1.predict(X_test)
mse1 = mean_squared_error(y_test,pred1)

pred2 = model2.predict(X_test)
mse2 = mean_squared_error(y_test,pred2)

pred3 = model3.predict(X_test)
mse3 = mean_squared_error(y_test,pred3)

pred4 = model4.predict(X_test)
mse4 = mean_squared_error(y_test,pred3)
print(mse1,mse2,mse3,mse4)

0.3464394873885773 0.5641519062500001 0.27611012194687534 0.27611012194687534
